**0. Septup**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score

**1. Cargar dataset**

In [2]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('insurance.csv')

# Mostrar las primeras filas del DataFrame para verificar
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


| Columna  | Significado                  |
| -------- | ---------------------------- |
| age      | Edad                         |
| sex      | Género                       |
| bmi      | Índice de masa corporal      |
| children | Número de hijos dependientes |
| smoker   | Si fuma o no                 |
| region   | Zona geográfica              |
| charges  | Costo médico anual           |


In [8]:
X = df.drop(columns="charges")
y = df["charges"]

numeric_cols = ["age", "bmi", "children"]
categorical_cols = ["sex", "smoker", "region"]

numeric_transformer = Pipeline([
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer([
    ("num", numeric_transformer, numeric_cols),
    ("cat", categorical_transformer, categorical_cols)
])

model = Pipeline([
    ("preprocess", preprocess),
    ("lr", LinearRegression())
])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("R²:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))

R²: 0.7672642952734356
MAE: 4243.654116653146


In [5]:
df["charges"].describe()

,charges
count,1338.000000
mean,13270.422265
std,12110.011237
min,1121.873900
25%,4740.287150
50%,9382.033000
75%,16639.912515
max,63770.428010


In [6]:
df.groupby("smoker")["charges"].mean()

,charges
smoker,
no,8434.268298
yes,32050.231832


**Validación cruzada**

In [9]:
from sklearn.model_selection import cross_val_score
import numpy as np

# R²
r2_scores = cross_val_score(model, X, y, cv=5, scoring="r2")
print("R² por fold:", r2_scores)
print("R² promedio:", r2_scores.mean())
print("Desviación estándar:", r2_scores.std())

# MAE
mae_scores = -cross_val_score(model, X, y, cv=5, scoring="neg_mean_absolute_error")
print("\nMAE por fold:", mae_scores)
print("MAE promedio:", mae_scores.mean())
print("Desviación estándar:", mae_scores.std())

R² por fold: [0.76148179 0.70649339 0.77806343 0.73269475 0.75557475]
R² promedio: 0.746861624347374
Desviación estándar: 0.024873741442236728

MAE por fold: [4239.55240265 4260.8069153  4015.77345487 4256.1702402  4216.31183933]
MAE promedio: 4197.722970468904
Desviación estándar: 92.30009870416129


**Analicemos el R²**

Promedio ≈ 0.747
Desviación estándar ≈ 0.025

¿Qué significa esto?

* El modelo es estable.
* La variabilidad entre folds es baja.

El rango es aproximadamente:
* Mínimo ≈ 0.706
* Máximo ≈ 0.778

Eso es un comportamiento muy razonable.

Desviación entre 0.02 - 0.03 es saludable.

Conclusión: el modelo generaliza de forma consistente.

**Analicemos el MAE**

MAE por fold: ~ 4.200 promedio
Desviación estándar ≈ 92

Eso es extremadamente estable.


Si el MAE promedio es 4.200 y la variación es solo 92, eso es alrededor de:

92/ 4200 ≈ 2%

Muy pequeño.

Eso es una señal fuerte de que el modelo no depende críticamente de la partición.

***El modelo explica aproximadamente el 75% de la variabilidad del gasto médico y comete un error promedio estable cercano a 4.200 dólares. La baja desviación entre folds indica que su desempeño es consistente.***